In [1]:
from multiprocessing.sharedctypes import Value
import networkx as nx
import numpy as np
import random
import pandas as pd

from tqdm import tqdm
from datetime import datetime
import pickle

# File imports
import data
from data import select_dataset, get_station_g, ingest_electricity_data, set_random_speed_columns
from replicate_graph import layer_graph
from vehicle import Vehicle
from simulation import Simulation
from visualization import set_draw_attributes

In [2]:
simulation_length = 12
battery_interval = 20
km_per_percent = 3.13
stations_path = "data/wcctci_stations-updated.csv"
distances_path = "data/wcctci_coord_distances.csv"
name = "wcctci"

number_of_iterations = 1


for n in range(0, number_of_iterations):
    
    ### run the simulation
    
    sim = Simulation(name, stations_path, distances_path, simulation_length, battery_interval, km_per_percent)
    sim.add_demand_nodes()
    sim.run()
    
    
    ### Print metrics
    
    #print name of this simulation
    print(name)
    print("station_utilization_disp_of_avg:  ", sim.metrics['station_utilization_disp_of_avg'])
    print("station_utilization_avg_of_disp:  ", sim.metrics['station_utilization_avg_of_disp'])
    print("electricity:  ", sim.metrics['electricity'])
    print("percent_delay:  ", sim.metrics['percent_delay'])
    print("hours_spent_in_queues:  ", sim.metrics['hours_spent_in_queues'])
    print("hours_spent_charging:  ", sim.metrics['hours_spent_charging'])
    print('____________________')
    
    
    ### get station utilization
    
    #get the station ids in order 
    current_stations = [int(j) for j in sim.station_g.nodes]

    #get the utilization rate of the stations
    utilization = sim.metrics['station_utilization']

    #get the station that has the highest utilization.
    h_st = current_stations[np.argmax(utilization)]

    #get the station that has the lowest utilization.
    l_st = current_stations[np.argmin(utilization)]
    
    
    ### Find the unused location that is closest to the highly utilized station
    
    #get the map of all parking areas and stations
    full_map_dist = pd.read_csv("data/charging_parking_distances.csv")
    full_map_loc = pd.read_csv("data/charging_parking_stations.csv")
    
    #keep only the roads that start or end with the high utilization station
    h_map = full_map_dist[(full_map_dist.loc[:, 'OriginID'] == h_st) | (full_map_dist.loc[:, 'DestinationID'] == h_st)]

    #then we eliminate all of the stations that are already being used.
    using_st = current_stations.copy()
    using_st.remove(h_st)
    h_map = h_map[(~ h_map.loc[:, 'OriginID'].isin(using_st)) & (~ h_map.loc[:, 'DestinationID'].isin(using_st))]

    #then we remove the case where we go from h_st to h_st
    h_map = h_map[~ ((h_map.loc[:, 'OriginID'] == h_st) & (h_map.loc[:, 'DestinationID'] == h_st))]

    #find the shortest edge. If there is more than one, we choose the first one that comes up (hence the 'reset_index' and [0] bellow)

    new_origin = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['OriginID'][0]
    new_destination = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['DestinationID'][0]

    #one of the two will be h_st and the other will be the new station location. 
    if new_origin == h_st:
        new_station = new_destination
    else:
        new_station = new_origin
      
    
    ### create the new maps with the locations and the distances of the stations that we will be using next
    
    new_stations_list = current_stations + [new_station]
    new_stations_list.remove(l_st)

    #now we reduce these to the new set of stations. All origins and destinations should be in this list.
    new_map_dist = full_map_dist[(full_map_dist.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_dist.loc[:, 'DestinationID'].isin(new_stations_list))]
    new_map_loc = full_map_loc[full_map_loc.loc[:, 'OID_'].isin(new_stations_list)]
   
    #we add the charging rates and capacity- assuming that they are the same as the others (set to 45 and 8)
    #TO DO: check this
    new_map_loc['charging_rate'] = 45
    new_map_loc['physical_capacity'] = 8
    
    #we change the snapX and snapY columns to lon and lat
    new_map_loc = new_map_loc.rename(columns = {'SnapX': 'longitude', 'SnapY': 'latitude'})
    
    #now we save them. 
    new_map_loc.to_csv("data/algorithm_"+str(n+1)+"_stations.csv") 
    new_map_dist.to_csv("data/algorithm_"+str(n+1)+"_distances.csv")
    

    #reset the variables for the next iteration
    stations_path = "data/algorithm_"+str(n+1)+"_stations.csv"
    distances_path = "data/algorithm_"+str(n+1)+"_distances.csv"
    name = "algorithm_"+str(n+1)
    

#we run the simulation for the final alocation

### run the simulation

sim = Simulation(name, stations_path, distances_path, simulation_length, battery_interval, km_per_percent)
sim.add_demand_nodes()
sim.run()


### Print metrics

#print name of this simulation
print(name)
print("station_utilization_disp_of_avg:  ", sim.metrics['station_utilization_disp_of_avg'])
print("station_utilization_avg_of_disp:  ", sim.metrics['station_utilization_avg_of_disp'])
print("electricity:  ", sim.metrics['electricity'])
print("percent_delay:  ", sim.metrics['percent_delay'])
print("hours_spent_in_queues:  ", sim.metrics['hours_spent_in_queues'])
print("hours_spent_charging:  ", sim.metrics['hours_spent_charging'])

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:25<00:00,  2.16s/it]


wcctci
station_utilization_disp_of_avg:   7.4078125
station_utilization_avg_of_disp:   7.409635416666666
electricity:   0
percent_delay:   (1.0, 1.0, 2.1620440424106673, 1.0560713883536805, 0.13829063132137182)
hours_spent_in_queues:   (0.0, 0.0, 3.6, 0.027795527156549524, 0.2578987276393834)
hours_spent_charging:   (0, 0.0, 21, 0.6190095846645367, 2.735587789237221)
____________________


<ipython-input-2-788fb58b9d10>:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_map_loc['charging_rate'] = 45
<ipython-input-2-788fb58b9d10>:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_map_loc['physical_capacity'] = 8
  0%|                                                                                           | 0/12 [00:00<?, ?it/s]


KeyError: 'speed_0'

## Bellow this is  just code I used to think about/ build the algorithm above.
### I was thinking about just one iteration, starting from the wcctci pickle.

In [2]:
#open the pickle
with open('trials/wcctci_04_04_2022_12_15_28.pkl', 'rb') as inp:
    res = pickle.load(inp)

Get the highest and lowest utilized stations:

In [7]:
#get station utilization

#get the station ids in order 
current_stations = list(res.station_g.nodes)

#get the utilization rate of the stations
utilization = res.metrics['station_utilization']

#get the station that has the highest utilization.
h_st = current_stations[np.argmax(utilization)]

#get the station that has the lowest utilization.
l_st = current_stations[np.argmin(utilization)]

Find the unused location that is closest to the highly utilized station:

In [41]:
#get the map of all parking areas and stations
full_map_dist = pd.read_csv("data/charging_parking_distances.csv")

#keep only the roads that start or end with the high utilization station
h_map = full_map_dist[(full_map_dist.loc[:, 'OriginID'] == h_st) | (full_map_dist.loc[:, 'DestinationID'] == h_st)]

#then we eliminate all of the stations that are already being used.
using_st = current_stations.copy()
using_st.remove(h_st)
h_map = h_map[(~ h_map.loc[:, 'OriginID'].isin(using_st)) & (~ h_map.loc[:, 'DestinationID'].isin(using_st))]

#then we remove the case where we go from h_st to h_st
h_map = h_map[~ ((h_map.loc[:, 'OriginID'] == h_st) & (h_map.loc[:, 'DestinationID'] == h_st))]

#find the shortest edge. If there is more than one, we choose the first one that comes up (hence the 'reset_index' and [0] bellow)

new_origin = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['OriginID'][0]
new_destination = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['DestinationID'][0]

#one of the two will be h_st and the other will be the new station location. 
if new_origin == h_st:
    new_station = new_destination
else:
    new_station = new_origin

Now we start to think about re-runing the simulation with this swap (taking out l_st and adding new_station). To do this, we need to create a new map with the locations and the distances of the stations that we will be using next.

In [ ]:
new_stations_list = current_stations + [new_station]

In [ ]:
#we have "full_map_dist" (distances map)
#and we need the locations map
full_map_loc = pd.read_csv("data/charging_parking_stations.csv")

#now we reduce these to the new set of stations. All origins and destinations should be in this list.
new_map_dist = full_map_dist[(full_map_dist.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_dist.loc[:, 'DestinationID'].isin(new_stations_list))]
new_map_loc = full_map_loc[(full_map_loc.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_loc.loc[:, 'DestinationID'].isin(new_stations_list))]

In [ ]:
#now we save them. 
new_map_loc.to_csv("data/algorithm_"+str(1)+"_stations.csv") ########change the 1
new_map_dist.to_csv("data/algorithm_"+str(1)+"_distances.csv")

Now we can run the next simulation

In [ ]:
simulation_length = 12
battery_interval = 20
km_per_percent = 3.13

sim = Simulation("data/algorithm_"+str(1)+"_stations.csv", "data/algorithm_"+str(1)+"_distances.csv", simulation_length, battery_interval, km_per_percent)